In [8]:
# importing the standard libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport

pd.set_option('display.max_columns', 500)

In [9]:
# math m and portugese p classes. Datasets taken from the UCI Machine Learning Repository 
p_orig = pd.read_csv('student_por.csv', sep=';')
m_orig = pd.read_csv('student_mat.csv', sep=';')

In [10]:
p_orig.head()

# many nominal/binary features ie Mothers Education (Medu) 1-5 for different degree levels etc

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,0,yes,no,no,no,yes,yes,yes,no,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,no,no,yes,yes,no,no,4,3,2,1,2,5,0,11,13,13


In [11]:
# data cleaning 
def clean_up(data):
    '''
    cleanup the dfs, using dummy variables for binary columns
    '''
    data.address = data.address.apply(lambda x: 'Urban' if x == 'U' else 'Rural')
    data = pd.get_dummies(data=data, columns=['address'], prefix='address_type', drop_first=True)
    data.famsize = data.famsize.apply(lambda x: 'BIG_FAMILY' if x == 'GT3' else 'SMALL_FAMILY') 
    data = pd.get_dummies(data=data, columns=['famsize'])
    data = data.drop(columns='famsize_SMALL_FAMILY')
    data.Pstatus = data.Pstatus.apply(lambda x: 'Together' if x == 'T' else 'Apart') 
    data = pd.get_dummies(data=data, columns=['Pstatus'], drop_first=True)
    data = pd.get_dummies(data=data, columns=['Mjob', 'Fjob'])
    data = data.drop(columns=['Mjob_other', 'Fjob_other'])
    data = pd.get_dummies(data=data, columns=['reason'])
    data = data.drop(columns=['reason_other'])
    data.schoolsup = data.schoolsup.apply(lambda x: 1 if x == 'yes' else 0) 
    data.famsup = data.famsup.apply(lambda x: 1 if x == 'yes' else 0) 
    data['tutor'] = data.paid.apply(lambda x: 1 if x == 'yes' else 0)
    data = data.drop(columns='paid')
    data.activities = data.activities.apply(lambda x: 1 if x == 'yes' else 0) 
    data.nursery = data.nursery.apply(lambda x: 1 if x == 'yes' else 0) 
    data.higher = data.higher.apply(lambda x: 1 if x == 'yes' else 0) 
    data.internet = data.internet.apply(lambda x: 1 if x == 'yes' else 0) 
    data.romantic = data.romantic.apply(lambda x: 1 if x == 'yes' else 0) 
    return data 

In [12]:
p = clean_up(p_orig)

In [14]:
m = clean_up(m_orig)

In [ ]:
from statsmodels.formula.api import ols
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression
from sklearn.feature_selection import RFECV
from sklearn.linear_model import Lasso